## Pre Setting

In [37]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkContext
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,lit
from pyspark.sql.types import StringType

from pyspark import SparkFiles

An error was encountered:
Invalid status code '404' from http://ip-172-31-7-69.ec2.internal:8998/sessions/1 with error payload: {"msg":"Session '1' not found."}


In [2]:
sc = SparkSession.builder.getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sqlContext = SQLContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Choose the columns with the largest number as "full column" -> "standard column"
# the file 'book800k-900k.csv' has all 20 columns, so we read it and save its columns as the benchmark for other files we are going to read 

full_col_df = spark.read.csv('s3://team10final/goodreads/cleansed data/book800k-900k.csv',header =True)
full_col_df=full_col_df.drop('_c0')
full_col_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Id: string, Name: string, Authors: string, ISBN: string, Rating: string, PublishYear: string, PublishMonth: string, PublishDay: string, Publisher: string, RatingDist5: string, RatingDist4: string, RatingDist3: string, RatingDist2: string, RatingDist1: string, RatingDistTotal: string, CountsOfReview: string, Language: string, pagesNumber: string, Description: string, Count of text reviews: string]

In [3]:
standard_col = full_col_df.columns
standard_col # check all columns we need

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'Name', 'Authors', 'ISBN', 'Rating', 'PublishYear', 'PublishMonth', 'PublishDay', 'Publisher', 'RatingDist5', 'RatingDist4', 'RatingDist3', 'RatingDist2', 'RatingDist1', 'RatingDistTotal', 'CountsOfReview', 'Language', 'pagesNumber', 'Description', 'Count of text reviews']

In [5]:
# load the data part1 to part 20 (corresponding to book100-200k to book1900k-2000k) (structured table name)
df = {}
for i in range(1,20):
    name=str(i)+"00"
    name_1 = str(i+1)+'00'
    file_name = 'book'+name+'k'+'-'+name_1+'k' # specify the file name we want to load at this iteration
    print(file_name)
    file = 's3://team10final/goodreads/cleansed data/'+file_name+'.csv'
    df[str(i)] = spark.read.option("delimiter", ",").csv(file,header=True)
    df[str(i)] = df[str(i)].drop('_c0')
    # create a empty column for a dataframe has less than 20 columns
    if len(df[str(i)].columns) <20:
        
        # iterate every column name all 20 columns
        for j in standard_col:
            if j not in df[str(i)].columns:
                df[str(i)] = df[str(i)].withColumn(str(j), lit(None)) # check missing column for each file, and create a empty column for those files having missing column
                
    df[str(i)].count() # check number of rows
    
    len(df[str(i)].columns) # check number of columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

book100k-200k
57046
20
book200k-300k
56182
20
book300k-400k
56586
20
book400k-500k
55155
20
book500k-600k
54859
20
book600k-700k
56827
20
book700k-800k
55913
20
book800k-900k
51480
20
book900k-1000k
42163
20
book1000k-1100k
41063
20
book1100k-1200k
43294
20
book1200k-1300k
44799
20
book1300k-1400k
39323
20
book1400k-1500k
35793
20
book1500k-1600k
34474
20
book1600k-1700k
33867
20
book1700k-1800k
33042
20
book1800k-1900k
39751
20
book1900k-2000k
44480
20

In [6]:
# load data part2 (unstructer for 'for loop')
df['0'] = spark.read.option("delimiter", ",").csv('s3://team10final/goodreads/cleansed data/book1-100k.csv',header=True,
                     inferSchema=True)
df['0'] = df['0'].drop('_c0')
df['20'] = spark.read.option("delimiter", ",").csv('s3://team10final/goodreads/cleansed data/book2000k-3000k.csv',header=True,
                     inferSchema=True)
df['20'] = df['20'].drop('_c0')
df['21'] = spark.read.option("delimiter", ",").csv('s3://team10final/goodreads/cleansed data/book3000k-4000k.csv',header=True,
                     inferSchema=True)
df['21'] = df['21'].drop('_c0')
df['22'] = spark.read.option("delimiter", ",").csv('s3://team10final/goodreads/cleansed data/book4000k-5000k.csv',header=True,
                     inferSchema=True)
df['22'] = df['22'].drop('_c0')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
for i in [0,20,21,22]:
    if len(df[str(i)].columns) <21:
        for j in standard_col:
            if j not in df[str(i)].columns:
                df[str(i)] = df[str(i)].withColumn(str(j), lit(None)) # create a empty column
    len(df[str(i)].columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20
20
20
20

In [8]:
# reorder the columns for each file in our dictionary for future concat
df['0'] = df['0'][standard_col]
df['1'] = df['1'][standard_col]
df['2'] = df['2'][standard_col]
df['3'] = df['3'][standard_col]
df['4'] = df['4'][standard_col]
df['5'] = df['5'][standard_col]
df['6'] = df['6'][standard_col]
df['7'] = df['7'][standard_col]
df['8'] = df['8'][standard_col]
df['9'] = df['9'][standard_col]
df['10'] = df['10'][standard_col]
df['11'] = df['11'][standard_col]
df['12'] = df['12'][standard_col]
df['13'] = df['13'][standard_col]
df['14'] = df['14'][standard_col]
df['15'] = df['15'][standard_col]
df['16'] = df['16'][standard_col]
df['17'] = df['17'][standard_col]
df['18'] = df['18'][standard_col]
df['19'] = df['19'][standard_col]
df['20'] = df['20'][standard_col]
df['21'] = df['21'][standard_col]
df['22'] = df['22'][standard_col]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.pop('17') # this file has really messy description for books which cannot be modified or cleansed, drop it directly for efficiency and integrity

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Id: string, Name: string, Authors: string, ISBN: string, Rating: string, PublishYear: string, PublishMonth: string, PublishDay: string, Publisher: string, RatingDist5: string, RatingDist4: string, RatingDist3: string, RatingDist2: string, RatingDist1: string, RatingDistTotal: string, CountsOfReview: string, Language: string, pagesNumber: string, Description: string, Count of text reviews: null]

In [12]:
# concat all the df we imported
df_final = df['0']
for i in range(16):
    df_final = df_final.union(df[str(i+1)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# concat all the df we imported
for i in range(17,len(df)-1):
    df_final = df_final.union(df[str(i+1)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_final.show(vertical=True) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-------------------------------------
 Id                    | 1                    
 Name                  | Harry Potter and ... 
 Authors               | J.K. Rowling         
 ISBN                  | null                 
 Rating                | 4.57                 
 PublishYear           | 2006                 
 PublishMonth          | 16                   
 PublishDay            | 9                    
 Publisher             | Scholastic Inc.      
 RatingDist5           | 5:1546466            
 RatingDist4           | 4:556485             
 RatingDist3           | 3:159960             
 RatingDist2           | 2:25317              
 RatingDist1           | 1:9896               
 RatingDistTotal       | total:2298124        
 CountsOfReview        | 28062                
 Language              | eng                  
 pagesNumber           | 652                  
 Description           | null                 
 Count of text reviews | null                 
-RECORD 1----

In [15]:
df_books=df_final#########tt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df_books.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- PublishYear: string (nullable = true)
 |-- PublishMonth: string (nullable = true)
 |-- PublishDay: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- RatingDist5: string (nullable = true)
 |-- RatingDist4: string (nullable = true)
 |-- RatingDist3: string (nullable = true)
 |-- RatingDist2: string (nullable = true)
 |-- RatingDist1: string (nullable = true)
 |-- RatingDistTotal: string (nullable = true)
 |-- CountsOfReview: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- pagesNumber: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Count of text reviews: string (nullable = true)

# Preprocess data Change DataType

## Preprocess Data


In [16]:
# get rid of ':' in order to create a numerical feature as the number of each rating
df_books = df_books.withColumn("RatingDist_1",F.split("RatingDist1",":")[1])
df_books = df_books.withColumn("RatingDist_2",F.split("RatingDist2",":")[1])
df_books = df_books.withColumn("RatingDist_3",F.split("RatingDist3",":")[1])
df_books = df_books.withColumn("RatingDist_4",F.split("RatingDist4",":")[1])
df_books = df_books.withColumn("RatingDist_5",F.split("RatingDist5",":")[1])
df_books = df_books.withColumn("RatingDist_total",F.split("RatingDistTotal",":")[1])
df_books.select("RatingDist_1","RatingDist_2","RatingDist_3","RatingDist_4","RatingDist_5","RatingDist_total").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+------------+------------+------------+----------------+
|RatingDist_1|RatingDist_2|RatingDist_3|RatingDist_4|RatingDist_5|RatingDist_total|
+------------+------------+------------+------------+------------+----------------+
|        9896|       25317|      159960|      556485|     1546466|         2298124|
|       12455|       37005|      211781|      604283|     1493113|         2358637|
|      108202|      130310|      567458|     1513191|     4268227|         6587388|
|       11896|       49353|      288821|      706082|     1504505|         2560657|
|       10128|       24849|      194848|      630534|     1749958|         2610317|
|        9419|       24282|      178419|      606800|     1612165|         2431085|
|         402|         283|        1201|        4650|       37432|           43968|
|           0|           5|           6|           7|          10|              28|
|         257|         218|        1074|        4358|       24406|          

In [17]:
# merge three time columns to one to convert to datetime
df_books = df_books.withColumn("Published_date", F.to_date(F.concat_ws("-","PublishYear","PublishDay","PublishMonth")))
df_books.select("Published_date").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|Published_date|
+--------------+
|    2006-09-16|
|    2004-09-01|
|    2003-11-01|
|    2003-11-01|
|    2004-05-01|
|    2002-09-28|
|    2004-09-13|
|    2005-04-26|
|    2005-09-12|
|    2005-11-01|
|    2002-04-30|
|    2004-08-03|
|    1996-01-17|
|    2004-09-14|
|    2002-12-03|
|    2004-09-14|
|    2001-05-15|
|    2000-06-28|
|    1990-08-28|
|    1993-03-28|
+--------------+
only showing top 20 rows

In [48]:

df_books

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Id: string, Name: string, Authors: string, ISBN: string, Rating: string, PublishYear: string, PublishMonth: string, PublishDay: string, Publisher: string, RatingDist5: string, RatingDist4: string, RatingDist3: string, RatingDist2: string, RatingDist1: string, RatingDistTotal: string, CountsOfReview: string, Language: string, pagesNumber: string, Description: string, Count of text reviews: string, RatingDist_1: string, RatingDist_2: string, RatingDist_3: string, RatingDist_4: string, RatingDist_5: string, RatingDist_total: string, Published_date: date]

## Change DataType

In [18]:
# convert the columns datae types to the ones we need for future analysis
# df_books.printSchema()
df_books = df_books.withColumn("Id",F.col("Id").cast("bigint"))
df_books = df_books.withColumn("CountsOfReview",F.col("CountsOfReview").cast("int"))
df_books = df_books.withColumn("Rating",F.col("Rating").cast("float"))
df_books = df_books.withColumn("pagesNumber",F.col("pagesNumber").cast("int"))
df_books = df_books.withColumn("RatingDist_1",F.col("RatingDist_1").cast("int"))
df_books = df_books.withColumn("RatingDist_2",F.col("RatingDist_2").cast("int"))
df_books = df_books.withColumn("RatingDist_3",F.col("RatingDist_3").cast("int"))
df_books = df_books.withColumn("RatingDist_4",F.col("RatingDist_4").cast("int"))
df_books = df_books.withColumn("RatingDist_5",F.col("RatingDist_5").cast("int"))
df_books = df_books.withColumn("RatingDist_total",F.col("RatingDist_total").cast("int"))
df_books = df_books.withColumn("CountsOfReview",F.col("CountsOfReview").cast("int"))

df_books_final = df_books.drop("RatingDist1","RatingDist2","RatingDist3","RatingDist4","RatingDist5","RatingDistTotal","PublishMonth","PublishDay","PublishYear")
df_books_final.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- CountsOfReview: integer (nullable = true)
 |-- Language: string (nullable = true)
 |-- pagesNumber: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- Count of text reviews: string (nullable = true)
 |-- RatingDist_1: integer (nullable = true)
 |-- RatingDist_2: integer (nullable = true)
 |-- RatingDist_3: integer (nullable = true)
 |-- RatingDist_4: integer (nullable = true)
 |-- RatingDist_5: integer (nullable = true)
 |-- RatingDist_total: integer (nullable = true)
 |-- Published_date: date (nullable = true)

In [20]:
df_books_final.createOrReplaceTempView("temp_table") # create a table in to query and save in the hive to read
spark.sql("create table book as select * from temp_table");

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [34]:
rt = spark.read.csv('s3://team10final/goodreads/rating_preprocessed.csv',header =True)
rt.createOrReplaceTempView("temp_table1")
spark.sql("create table rating as select * from temp_table1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [21]:
# 1. What is the distribution of books across languages?
df1=spark.sql("select distinct Language,count(*) as Num_of_books FROM book GROUP BY Language order by Num_of_books desc limit 20")
df1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|Language|Num_of_books|
+--------+------------+
|    null|     1337073|
|     eng|      167024|
|   en-US|       20510|
|     fre|       14394|
|     ger|        9832|
|   en-GB|        7271|
|     spa|        6470|
|     jpn|        1776|
|     ita|         953|
|     mul|         375|
|     por|         355|
|      nl|         300|
|     rus|         253|
|     swe|         219|
|     zho|         198|
|     pol|         168|
|     per|         124|
|     ind|         119|
|     lat|         100|
|     grc|          93|
+--------+------------+

In [24]:
# 2. Which are the top 10 most rated books?
df2=spark.sql("SELECT distinct Name, AVG(Rating) as rate_of_book FROM book GROUP BY Name ORDER BY rate_of_book DESC LIMIT 20")
df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|                Name|rate_of_book|
+--------------------+------------+
|Ibm Hacmp For Aix...|         5.0|
|Ashton Kutcher: T...|         5.0|
|    América indígena|         5.0|
|Shock Rattle & Ro...|         5.0|
|Birnbaum's Walt D...|         5.0|
|fifty-one years: ...|         5.0|
|Great Art Treasur...|         5.0|
|       Robert Hayden|         5.0|
|Contemporary Anti...|         5.0|
|Cribbet and Johns...|         5.0|
|Markedness: The E...|         5.0|
|Renaissance to Ro...|         5.0|
|The Old Man and t...|         5.0|
|Good Clinical Pra...|         5.0|
|Performance by De...|         5.0|
|The Oyster and th...|         5.0|
|Iran Hizbullah Ha...|         5.0|
|The Redemption of...|         5.0|
|The Pilot's Daughter|         5.0|
|Unto the Last Gen...|         5.0|
+--------------------+------------+

In [25]:
# 4. Which are the authors with most books?

df3=spark.sql("SELECT Authors, count(*) AS num_of_book FROM book GROUP BY Authors ORDER BY num_of_book desc limit 20")
df3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|             Authors|num_of_book|
+--------------------+-----------+
|                null|      30632|
|           Anonymous|       2616|
|             Unknown|       1613|
| William Shakespeare|       1232|
|     Francine Pascal|        906|
|Cram101 Textbook ...|        861|
|     Agatha Christie|        835|
|Fodor's Travel Pu...|        797|
|        Harold Bloom|        725|
|        Isaac Asimov|        661|
|       Carolyn Keene|        629|
|             Various|        628|
|        Nora Roberts|        619|
|          R.L. Stine|        607|
|        Stephen King|        603|
|Hal Leonard Publi...|        579|
|National Research...|        575|
| Walt Disney Company|        536|
|          NOT A BOOK|        516|
|       Ann M. Martin|        498|
+--------------------+-----------+

In [33]:
# 5.Which are the top 10 highly rated authors?


df4=spark.sql("SELECT Distinct Authors, AVG(Rating) as rate_of_book FROM book GROUP BY Authors ORDER BY rate_of_book DESC LIMIT 10")
df4.show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+
|            Authors|rate_of_book|
+-------------------+------------+
|     Kenneth Dorter|         5.0|
| Larissa P. Watkins|         5.0|
|Stacy Rae Rubalcaba|         5.0|
| Rebecca Lynn Winer|         5.0|
|     C. Wayne Owens|         5.0|
|      David  Wolman|         5.0|
|         Val Farmer|         5.0|
|       Arthur Bowen|         5.0|
|     John Breadsley|         5.0|
|     B. Wayne Quist|         5.0|
+-------------------+------------+

In [31]:
df_books_final.write.option("delimiter", ",").option("header","true").csv("s3://team10final/goodreads/final")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…